In [ ]:
!pip install mne

In [ ]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mne

In [ ]:
import glob
import getpass
import importlib
import functions
import spectrogram_plotting_functions
import scipy.signal
importlib.reload(functions)
importlib.reload(spectrogram_plotting_functions)

In [ ]:

user= (getpass.getuser())
print("Hello", user)

base='C:\\Users\\{}\\Dropbox\\CPLab'.format(user)
files = glob.glob(base+'\\all_data_mat\\*.mat')
savepath = base+'\\results\\'
print(files)

In [ ]:
keyboard_dict={'98':'b','119':'w','120':'nc','49':'1','48':'0'}

In [ ]:
import os
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
files=['C:\\Users\\Thomas\\Dropbox\\CPLab\\all_data_mat_filtered\\20230529_dk1_nocontext.mat']
event_data_df=[]
shuffled_event_data_df=[]
events_codes_all = {}
before_data_power_df=pd.DataFrame(columns=['rat_id','task','channel_id','before_data_power','before_data_spec'])
scrambled_baseline_power_df=pd.DataFrame(columns=['rat_id','task','channel_id','before_data_power','before_data_spec'])
averaged_data=pd.DataFrame(columns=['rat_id','task','channel_id','average_power_spectrum'])
averaged_data_welch=pd.DataFrame(columns=['rat_id','task','channel_id','average_power_spectrum'])
random_baseline_data=[]
for file in files:
    #if 'dk6' in file:
        
        print(file)
        base_name = os.path.basename(file)
        base_name, _ = os.path.splitext(base_name)

        date, rat_id, task = functions.exp_params(base_name)
        print(date, rat_id, task)
        if task == 'nocontextday2' or task == 'nocontextos2':
            task = 'nocontext'
        print(date, rat_id, task)

        f = h5py.File(file, 'r')
        channels = list(f.keys())
        print(channels)

        if 'Keyboard' in channels:
            events = f['Keyboard']
        elif 'keyboard' in channels:
            events = f['keyboard']
        elif 'memory' in channels:
            events = f['memory']
        elif 'Memory' in channels:
            events = f['Memory']

        events_codes = np.array(events['codes'][0])
        events_times = np.array(events['times'][0])
        events_codes_all[base_name] = events_codes
        epochs = functions.generate_epochs_with_first_event(events_codes, events_times)
        
        mne_epoch_door_before=np.zeros((len(epochs),len(channels),4000))
        mne_epoch_door_after=np.zeros((len(epochs),len(channels),4000))
        mne_epoch_dig_before=np.zeros((len(epochs),len(channels),4000))
        mne_epoch_dig_after=np.zeros((len(epochs),len(channels),4000))
        first_event = events_times[0]
        
        for channel_num,channeli in enumerate(channels):
            if "AON" in channeli or "vHp" in channeli:
                channel_id = channeli
                data_all = f[channeli]
                raw_data = np.array(data_all['values']).flatten()
                raw_time = np.array(data_all['times']).flatten()
                sampling_rate = int(1 / data_all['interval'][0][0])
                print(raw_data.shape, raw_time.shape, sampling_rate)
                notch_filtered_data = functions.iir_notch(raw_data, sampling_rate, 60)

                normalized_data, time, data_before = functions.data_normalization(notch_filtered_data, raw_time, first_event, sampling_rate)
                
                # This makes a dataframe for calculating baseline power 
                bd_freq, bd_power = scipy.signal.welch(data_before,fs=2000.0, nperseg=512)
                #before_data_power=functions.calculate_power_1D(data_before)
                
                #before_data_power = np.sum(bd_power) * delta_f
                
                def band_powers(data, freqs):
                    delta_f = freqs[1] - freqs[0]

                    theta = np.where((freqs >= 4) & (freqs <= 12))
                    beta = np.where((freqs >= 12) & (freqs <= 30))
                    gamma = np.where((freqs >= 30) & (freqs <= 100))
                    total_power = np.sum(data)*delta_f
                    theta_power = np.sum(data[theta])*delta_f
                    beta_power = np.sum(data[beta])*delta_f
                    gamma_power = np.sum(data[gamma])*delta_f
                    return total_power, theta_power, beta_power, gamma_power
                
                before_data_power, baseline_theta_power, baseline_beta_power, baseline_gamma_power = band_powers(bd_power, bd_freq)
                before_data_row=pd.DataFrame({'rat_id':[rat_id],'task':[task],'channel_id':[channel_id],'before_data_power':[before_data_power],'before_data_spec':[bd_power],'beta_power':[baseline_beta_power],'theta_power':[baseline_theta_power],'gamma_power':[baseline_gamma_power]})
                before_data_power_df=pd.concat([before_data_power_df,before_data_row],ignore_index=True)
                
                scrambled_baseline_data=np.copy(data_before)
                #This makes a dataframe for calculating scrambled baseline power
                np.random.shuffle(scrambled_baseline_data)
                scrambled_baseline_freq, scrambled_baseline_power = scipy.signal.welch(scrambled_baseline_data,fs=2000.0, nperseg=512)
                scrambled_baseline_total_power, scrambled_baseline_theta_power, scrambled_baseline_beta_power, scrambled_baseline_gamma_power = band_powers(scrambled_baseline_power, scrambled_baseline_freq)
                scrambled_baseline_row = pd.DataFrame({'rat_id':[rat_id],'task':[task],'channel_id':[channel_id],
                                                       'before_data_power':[scrambled_baseline_total_power],'before_data_spec':[scrambled_baseline_power],
                                                       'beta_power':[scrambled_baseline_beta_power],'theta_power':[scrambled_baseline_theta_power],
                                                       'gamma_power':[scrambled_baseline_gamma_power]})
                scrambled_baseline_power_df=pd.concat([scrambled_baseline_power_df,scrambled_baseline_row],ignore_index=True)
                
                
                ##Calculate the power spectrum around events for each trial

                total = normalized_data
                compiled_data = []
                power_spectra = []
                
                compiled_data_welch = []
                power_spectra_welch = []

                
                for i, epochi in enumerate(epochs):
                    door_timestamp = epochi[0][0]
                    trial_type = epochi[0][1]
                    dig_type = epochi[1, 1]
                    print(dig_type)
                    dig_timestamp = epochi[1, 0]
                    print(door_timestamp, trial_type, dig_timestamp, dig_type)
                    data_complete_trial = functions.extract_complete_trial_data(total, time, door_timestamp, dig_timestamp, sampling_rate)
                    #Here we will extract and store the events data
                    data_door_before,data_door_after=functions.extract_door_data(total, time, door_timestamp, sampling_rate)    
                    data_dig_before, data_dig_after=functions.extract_dig_data(total,time, dig_timestamp, sampling_rate)
                    mne_epoch_door_before[i,channel_num,:]=list(data_door_before)
                    mne_epoch_door_after[i,channel_num,:]=list(data_door_after)
                    mne_epoch_dig_before[i,channel_num,:]=list(data_dig_before)
                    mne_epoch_dig_after[i,channel_num,:]=list(data_dig_after)

                    event_data_list=[data_door_before,data_door_after,data_dig_before,data_dig_after]
                    event_time=np.linspace(0,2,2*sampling_rate)
                    event_row=[]
                    shuffled_event_row=[]

                    event_row.extend([rat_id, task, channeli,before_data_power, baseline_beta_power, baseline_theta_power, baseline_gamma_power, i])
                    shuffled_event_row.extend([rat_id, task, channeli,scrambled_baseline_total_power, scrambled_baseline_beta_power, scrambled_baseline_theta_power, scrambled_baseline_gamma_power, i])
                    
                    for event_data in event_data_list:
                        faxis, Sxx_event = scipy.signal.welch(event_data,sampling_rate,nperseg=1024)
                        event_row.append(np.abs(Sxx_event))
                        total_power, theta_power, beta_power, gamma_power = band_powers(Sxx_event, faxis)
                        event_row.extend([total_power,beta_power, theta_power, gamma_power])
                        
                        #shuffled data
                        shuffled_data=np.copy(event_data)
                        np.random.shuffle(shuffled_data)
                        faxis_shuffled, Sxx_event_shuffled = scipy.signal.welch(shuffled_data,sampling_rate,nperseg=1024)
                        total_power_shuffled, theta_power_shuffled, beta_power_shuffled, gamma_power_shuffled = band_powers(Sxx_event_shuffled, faxis_shuffled)
                        shuffled_event_row.append(np.abs(Sxx_event_shuffled))
                        shuffled_event_row.extend([total_power_shuffled,beta_power_shuffled, theta_power_shuffled, gamma_power_shuffled])
                        

                    event_data_df.append(event_row)
                    shuffled_event_data_df.append(shuffled_event_row)


event_data_compiled_df = pd.DataFrame(event_data_df, columns=['rat_id','task','channel_id','baseline total','baseline beta','baseline theta','baseline gamma','trial','door before','door before total','door before beta','door before theta','door before gamma', 'door after','door after total','door after beta','door after theta','door after gamma','dig before','dig before total','dig before beta','dig before theta','dig before gamma','dig after','dig after total','dig after beta','dig after theta','dig after gamma'])
shuffled_event_data_compiled_df=pd.DataFrame(shuffled_event_data_df, columns=['rat_id','task','channel_id','baseline total','baseline beta','baseline theta','baseline gamma','trial','door before','door before total','door before beta','door before theta','door before gamma', 'door after','door after total','door after beta','door after theta','door after gamma','dig before','dig before total','dig before beta','dig before theta','dig before gamma','dig after','dig after total','dig after beta','dig after theta','dig after gamma'])
#shuffled_event_data_compiled_df = pd.DataFrame(shuffled_event_data_df, columns=['rat_id','task','channel_id','baseline total','baseline beta','baseline theta','baseline gamma','trial','door before','door before beta','door before theta','door before gamma', 'door after','door after beta','door after theta','door after gamma','dig before','dig before beta','dig before theta','dig before gamma','dig after','dig after beta','dig after theta','dig after gamma'])

In [ ]:
%matplotlib qt

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_reshaped = mne_epoch_door_before.reshape(-1, 4000)  # Reshape to (n_epochs * n_channels, n_times)
data_normalized = scaler.fit_transform(data_reshaped)
data_normalized = data_normalized.reshape(len(epochs), len(channels), 4000)  # Reshape back to (n_epochs, n_channels, n_times)

info = mne.create_info(ch_names=channels, sfreq=2000.0, ch_types='misc')



In [ ]:
# Create MNE EpochsArray
mne_epoch_door_before = mne.EpochsArray(data_normalized, info)

# Plot the data
fig = mne_epoch_door_before.plot(picks=np.arange(1, 8), n_epochs=10)


fig.show()

# info=mne.create_info(ch_names=channels,sfreq=2000.0,ch_types='misc')
# mne_epoch_door_before=mne.EpochsArray(mne_epoch_door_before,info)
# #fig,axs=plt.subplots(1,1,figsize=(30,10))
# fig=mne_epoch_door_before.plot(picks=np.arange(1,8),n_epochs=10)
# fig.show()

In [ ]:
!pip install mne_connectivity

In [ ]:
import mne
import mne_connectivity
fs=2000
freqs = np.arange(1,100)
n_cycles = freqs/2
con = mne_connectivity.spectral_connectivity_epochs(mne_epoch_door_before, method='coh', sfreq=int(fs),
                                                   mode='cwt_morlet', cwt_freqs=freqs,
                                                   cwt_n_cycles=n_cycles, verbose=True,
                                                   indices=None)

In [ ]:
%matplotlib inline
coh = con.get_data()
print(coh.shape)
plt.imshow(np.squeeze(coh[13,:,:]), extent=[0, 2, freqs[0], freqs[-1]],
           aspect='auto', origin='lower', cmap='jet')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.show()

In [ ]:
for i in range(0,coh.shape[0]):
    plt.imshow(np.squeeze(coh[i,:,:]), extent=[0, 2, freqs[0], freqs[-1]],
           aspect='auto', origin='lower', cmap='jet')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.title('Channel {}'.format(i))
    plt.show()